# Guided Exercises

## Goals:

- Read data
- Write data
- View and explore data 
- Peform basic calculations

## Read, Write, and View the Data

We can use dbutils to view the data available for this workshop.

In [0]:
data_path = "abfss://attributes@sa8451ccnthwstext.dfs.core.windows.net/"

display(dbutils.fs.ls(data_path))

Let's take a look inside the transaction data directory. 
1. How many files are there? 
2.  What are the file extensions? 
3.  What is the advantage of splitting the data into multiple small files vs. one large file, like a single csv?

In [0]:
display(dbutils.fs.ls(f"{data_path}/transaction_data"))

### Parquet Files

- Binary file that contains meta data about its contents
   - Embedded schema in the file
- Columnar data format
  - <img src ='https://www.dremio.com/wp-content/uploads/2022/04/chart-1024x412.png' width=700>
  - Not human readable
  - Reads much more efficiently
- Why is parquet a better choice than CSV?
  - Parquet’s *Write Once Read Many* paradigm
    - Slow to write but incredibly fast to read, especially when you’re only accessing a subset of the total columns
  - CSV is row-based and uses eager evaluation
    - Reads and parses entire dataset before performing transformations
    - More costly and less efficient for bigger jobs

<img src ='https://media-exp1.licdn.com/dms/image/C5612AQHDkeQFY4ujZQ/article-inline_image-shrink_1500_2232/0/1568305562015?e=1665619200&v=beta&t=M9UPgyesA1vKiteJjTXKg7OR8DpRlhOd9yQCL-sTuuw' width=800>

More Resources:
- https://www.linkedin.com/pulse/spark-file-format-showdown-csv-vs-json-parquet-garren-staubli/
- https://www.dremio.com/resources/guides/intro-apache-parquet/
- https://umbertogriffo.gitbook.io/apache-spark-best-practices-and-tuning/storage/use-the-best-data-format
- https://luminousmen.com/post/big-data-file-formats

### Partitioning

- **Partitioning** is the splitting of data into smaller, more manageable chunks
- Data can be partitioned both:
  - On disk (where the files are saved in the file system)
  - In memory (data loaded into Spark itself as a DataFrame)
- Saved data is automatically partitioned into multiple files by Spark
  - Allows for transformations to be executed in parallel
- Data can be partitioned based on the value in a certain column
  - This column is called the **Partition Key**
  - Loading data that is partitioned by a column that you’re going to filter on can make loading data *much* faster

### Reading Parquet Files

`read.parquet()` reads in the data from a parquet file. The result of loading a parquet file is a Spark DataFrame.

In [0]:
transaction_data = spark.read.parquet(f"{data_path}/transaction_data/")

### Writing Parquet Files

`write.parquet()` writes out the data in parquet file form

In [0]:
results_container =  "abfss://results@sa8451ccnthwstext.dfs.core.windows.net"

## REPLACE WIH YOUR EMAIL ###
user = "your.email@gmail.com"
#############################

output_path = f"{results_container}/{user}/transaction_data_output"
print(output_path)

### UNCOMMENT BELOW WHEN READY TO WRITE ##########
# transaction_data.write.mode("overwrite").parquet(output_path)

In [0]:
display(dbutils.fs.ls(output_path))

`partitionBy("partitionKey")` partitions the data by the specified partition key (i.e. a column in the dataframe) when writing

**Why do this?** Allows for quicker reads when restricting to certain parition key values

**What's the drawback?** You'll spend more time writing now than before (because of the data shuffling). But it'll save you so much time on reads!

In [0]:
output_partitioned_path = f"{results_container}/{user}/transaction_data_output_by_week"
print(output_partitioned_path)

transaction_data.write.partitionBy("WEEK_NO").mode("overwrite").parquet(output_partitioned_path)

Let's look at those partitions!

In [0]:
display(dbutils.fs.ls(output_partitioned_path))

### Print Schema

`printSchema()` displays the schema of the dataframe. `printSchema()` is a PySpark function.

In [0]:
transaction_data.printSchema()

### Display/Show

Use `display()` or `show()` to display a preview of the records in the dataframe. Keep in mind that `display()` is a Databricks specific function whereas `show()` is a PySpark specific function that can be used anywhere (i.e. in the terminal).

In [0]:
transaction_data.display()

In [0]:
transaction_data.show()

## Explore the Data

Examples of stuff we can cover:

- How many records are there?
- What are some metrics of the data?
- How many distinct records are there?

### Describe

`describe()` displays some metrics about the dataframe. `describe()` is a PySpark function.

In [0]:
transaction_data.describe().display()

### Count

`count()` returns the number of records in the  dataframe. `count()` is a PySpark function.

In [0]:
transaction_data.count()

### Select

`select()` narrows down the dataframe to only the specified columns. `select()` is a PySpark function.

In [0]:
transaction_data.select("household_key", "BASKET_ID", "PRODUCT_ID").display()

### Distinct

`distinct()` removes any duplicate records from the dataframe and returns the dataframe with only unique records. You **cannot** choose a subset of columns to perform the `distinct()` on.

In [0]:
transaction_data.distinct().display()

In [0]:
transaction_data.select("household_key", "BASKET_ID").distinct().display()

In [0]:
transaction_data.select("household_key", "BASKET_ID").distinct().count()

### Drop Duplicates

`dropDuplicates()` removes any duplicate records from the dataframe and returns the dataframe with only unique records. You **can** choose a subset of columns to perform the `dropDuplicates()` on while still returning all columns in the dataframe.

In [0]:
transaction_data.dropDuplicates().display()

In [0]:
transaction_data.dropDuplicates(["household_key", "BASKET_ID"]).display()

In [0]:
transaction_data.dropDuplicates(["household_key", "BASKET_ID"]).count()

## Questions

1.  How many records are in the coupon data?
2.  What's the min/max of each of the columns in the coupon data?
3.  How many unique coupon records are there?
4.  How many unique products are in the coupon data?

ANSWER TO QUESTION 1 - How many records are in the coupon data?

In [0]:
# QUESTION 1: Read in the coupon data
coupon = spark.read.parquet(f"{data_path}/coupon/")

In [0]:
# QUESTION 1: Perform a count to get the number of records in the coupon data
coupon.count()

ANSWER TO QUESTION 2 - What's the min/max of each of the columns in the coupon data?

In [0]:
# QUESTION 2: Perform a describe to easily find the min/max of each of the columns in the coupon data
coupon.describe().display()

ANSWER TO QUESTION 3 - How many unique coupon records are there?

In [0]:
# QUESTION 3: Method 1 - Perform a distinct to find the number of unique coupon records
coupon.distinct().count()

In [0]:
# QUESTION 3: Method 2 - Perform a dropDuplicates without specifying a subset to find the number of unique coupon records
coupon.dropDuplicates().count()

ANSWER TO QUESTION 4 - How many unique products are in the coupon data?

In [0]:
# QUESTION 4: Method 1 - Perform a select to get only the PRODUCT_ID column, then perform a distinct to get the number of unique products
coupon.select("PRODUCT_ID").distinct().count()

In [0]:
# QUESTION 4: Method 2 - Perform a dropDuplicates on PRODUCT_ID subset to get the number of unique products
coupon.dropDuplicates(["PRODUCT_ID"]).count()